In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from datetime import timedelta, datetime

# ==========================================
# 0. ダミーデータの生成
# (お手元のデータがある場合はここをスキップし、CSVを読み込んでください)
# ==========================================
def generate_suzuri_dummy_data(n_users=1000, n_creators=50, n_transactions=5000):
    np.random.seed(42)
    
    # --- 1. クリエイターテーブル作成 ---
    creators = []
    for i in range(n_creators):
        creators.append({
            'creator_id': i,
            'name': f'creator_{i}',
            'display_name': f'Display_Creator_{i}',
            'created_at': '2020-01-01',
            'official': np.random.choice([True, False], p=[0.1, 0.9]),
            'bio': 'Sample bio...'
        })
    df_creators = pd.DataFrame(creators)
    
    # クリエイターごとの「画風・世界観」ベクトル (128次元) を擬似的に作成
    # 実際は material_url の画像をCNNに通して得られるベクトルなどを想定
    creator_visual_features = np.random.rand(n_creators, 128)

    # --- 2. トランザクションデータ作成 ---
    data = []
    actions = ['view', 'add_to_cart', 'purchase', 'favorite']
    categories = ['T-shirt', 'Hoodie', 'Sticker', 'Smartphone Case', 'Tote Bag']
    
    start_date = datetime(2025, 1, 1)
    
    for _ in range(n_transactions):
        user_id = np.random.randint(0, n_users)
        
        # Whale (太客) バイアス: ID 0~9 は購入頻度が高く、同じものを買いがち
        is_whale = user_id < 10
        
        # アクション決定 (Whaleはpurchase率が高いとする)
        if is_whale:
            action = np.random.choice(actions, p=[0.1, 0.1, 0.7, 0.1])
        else:
            action = np.random.choice(actions, p=[0.5, 0.2, 0.1, 0.2])
            
        # 日時 (ランダムな1年間)
        days_offset = np.random.randint(0, 365)
        accessed_at = start_date + timedelta(days=days_offset)
        month = accessed_at.month
        
        # クリエイターとアイテム決定
        creator_id = np.random.randint(0, n_creators)
        product_id = f"prod_{creator_id}_{np.random.randint(0, 5)}"
        
        # カテゴリ決定 (季節性を反映)
        if 4 <= month <= 8: # 夏
            cat_name = np.random.choice(categories, p=[0.5, 0.1, 0.2, 0.1, 0.1])
        else: # 冬
            cat_name = np.random.choice(categories, p=[0.1, 0.5, 0.2, 0.1, 0.1])
            
        # 価格決定
        price_map = {'T-shirt': 3000, 'Hoodie': 5000, 'Sticker': 500, 'Smartphone Case': 2500, 'Tote Bag': 2000}
        price = price_map[cat_name]
        
        # Whaleは大量買いする (購入の場合、同じログを複数回入れるか、売上計算で考慮)
        # ここではシンプルに1行1購入とし、Whaleは何回もループを回っているとする
        
        row = {
            'user_id': user_id,
            'accessed_at': accessed_at, # datetime型
            'event_action': action,
            'product_id': product_id,
            'creator_name': f'creator_{creator_id}',
            'creator_id': creator_id,
            'title': f'Title {product_id}',
            'description': 'desc',
            'item_id': f'item_{product_id}',
            'item_name': f'Item {cat_name}',
            'item_category_id': 999,
            'item_category_name': cat_name,
            'exemplary_item_color_id': 1,
            'exemplary_item_color_name': 'White',
            'material_1': 123, # material ID
            'material_url': 'http://example.com/img.jpg',
            'sale_1': 0, # 簡略化
            'profit': price * 0.1,
            'price': price
        }
        data.append(row)
        
    df = pd.DataFrame(data)
    
    return df, df_creators, creator_visual_features

# データを生成
df, df_creators, creator_features = generate_suzuri_dummy_data()

# ==========================================
# 前処理: データの型変換とフィルタリング
# ==========================================
# 実際のCSV読み込み時はここで to_datetime 変換を行ってください
df['accessed_at'] = pd.to_datetime(df['accessed_at'])

# 分析用データの作成 (購入ログのみ)
df_purchase = df[df['event_action'] == 'purchase'].copy()
print(f"Total Transactions: {len(df)}")
print(f"Total Purchases: {len(df_purchase)}")


# ==========================================
# 分析1: Whale汚染度の検証 (売上金額 vs 購入人数)
# ==========================================
print("\n--- [Analysis 1] Whale Impact Check ---")

# プロダクトごとの「総売上金額」と「ユニーク購入者数(UU)」を集計
item_stats = df_purchase.groupby('product_id').agg(
    total_revenue=('price', 'sum'),
    unique_users=('user_id', 'nunique')
).reset_index()

# Top 30 ランキングの作成
top_revenue = item_stats.sort_values('total_revenue', ascending=False).head(30)['product_id'].values
top_uu = item_stats.sort_values('unique_users', ascending=False).head(30)['product_id'].values

# ランキングの乖離率 (Jaccard Distance)
intersection = len(set(top_revenue) & set(top_uu))
overlap_rate = intersection / 30
churn_rate = 1.0 - overlap_rate

print(f"Top 30 Overlap Rate (金額ランク vs 人数ランク): {overlap_rate:.2f}")
print(f"Ranking Churn Rate (入れ替わり率): {churn_rate:.2f}")

# ジニ係数の計算 (アイテムごとの購入者の偏り)
# 特定のアイテムの購入者分布を見て、少数が買い占めていないか確認
# ※ここでは全アイテム平均の簡易チェックを行う
def calc_gini(array):
    array = array.flatten()
    if np.amin(array) < 0: return -1 # マイナス値は不可
    array = np.sort(array)
    index = np.arange(1, array.shape[0] + 1)
    n = array.shape[0]
    return ((np.sum((2 * index - n - 1) * array)) / (n * np.sum(array)))

# 最も売上の高いアイテムを抽出してジニ係数を計算
top_item = item_stats.sort_values('total_revenue', ascending=False).iloc[0]['product_id']
top_item_buyers = df_purchase[df_purchase['product_id'] == top_item].groupby('user_id')['price'].sum().values
gini_score = calc_gini(top_item_buyers) if len(top_item_buyers) > 1 else 0

print(f"Gini Coefficient of Top Item ('{top_item}'): {gini_score:.2f}")

if churn_rate >= 0.3 or gini_score > 0.6:
    print(">> JUDGMENT: GO (一部のユーザーによる売上依存度が高く、Whale対策が必要です)")
else:
    print(">> JUDGMENT: CAUTION (売上は比較的広く分散しています)")


# ==========================================
# 分析2: 探索のポテンシャル (クリエイター類似度 vs 併売数)
# ==========================================
print("\n--- [Analysis 2] Exploration Potential ---")

# 1. クリエイター間の併売行列 (Co-occurrence)
# user_id x creator_id の行列を作る
user_creator_matrix = pd.crosstab(df_purchase['user_id'], df_purchase['creator_id'])
user_creator_matrix[user_creator_matrix > 0] = 1 # 購入有無(0/1)にする
co_occurrence_matrix = user_creator_matrix.T.dot(user_creator_matrix)

# 2. クリエイター間の画像類似度 (本来は material_url の画像解析結果を使用)
visual_sim_matrix = cosine_similarity(creator_features)

# 3. データの結合 (類似度と併売数のペアを作る)
plot_data = []
creators_list = list(co_occurrence_matrix.index)
# 計算量削減のためサンプリングまたはループ最適化推奨。ここでは全ペア実施
for i in range(len(creators_list)):
    for j in range(i + 1, len(creators_list)):
        c1, c2 = creators_list[i], creators_list[j]
        sim = visual_sim_matrix[c1, c2] # 画像類似度
        co_count = co_occurrence_matrix.iloc[i, j] # 併売数
        plot_data.append({'similarity': sim, 'co_purchase': co_count})

df_plot = pd.DataFrame(plot_data)

# 類似度ごとにビン分割して、平均併売数を計算
df_plot['sim_bin'] = pd.cut(df_plot['similarity'], bins=10)
bin_stats = df_plot.groupby('sim_bin', observed=False)['co_purchase'].mean()

print("Similarity Bins vs Avg Co-purchase:")
print(bin_stats)

# 可視化
plt.figure(figsize=(10, 5))
bin_centers = np.arange(0.05, 1.05, 0.1)
plt.plot(bin_centers, bin_stats.values, marker='o', linestyle='-')
plt.title('Visual Similarity vs Co-Purchase Count')
plt.xlabel('Similarity (Low -> High)')
plt.ylabel('Avg Co-Purchases')
plt.grid(True)
plt.show()

# 判断ロジック: 中程度の類似度(0.4-0.7)の山があるか？
mid_sim_avg = bin_stats.iloc[4:7].mean() # 0.4~0.7
high_sim_avg = bin_stats.iloc[8:].mean() # 0.8~1.0
if mid_sim_avg > high_sim_avg:
    print(">> JUDGMENT: GO (『似すぎない』クリエイター同士の併売が多く、探索レコメンドの余地があります)")
else:
    print(">> JUDGMENT: NEUTRAL (類似度と併売数が比例、またはランダムです)")


# ==========================================
# 分析3: LTV/離脱率のインパクト (金額ベース)
# ==========================================
print("\n--- [Analysis 3] LTV Impact (Single vs Multi Creator) ---")

# ユーザーごとの統計
user_stats = df_purchase.groupby('user_id').agg(
    unique_creators=('creator_id', 'nunique'),
    ltv=('price', 'sum')
).reset_index()

# グループ分け
single_creator_users = user_stats[user_stats['unique_creators'] == 1]
multi_creator_users = user_stats[user_stats['unique_creators'] >= 2]

ltv_single = single_creator_users['ltv'].mean()
ltv_multi = multi_creator_users['ltv'].mean()

uplift = ltv_multi / ltv_single if ltv_single > 0 else 0

print(f"Avg LTV (Single Creator): ¥{ltv_single:,.0f}")
print(f"Avg LTV (Multi Creator):  ¥{ltv_multi:,.0f}")
print(f"LTV Uplift: {uplift:.2f}x")

if uplift >= 1.5:
    print(">> JUDGMENT: GO (複数クリエイター推しのユーザーはLTVが圧倒的に高いです)")
else:
    print(">> JUDGMENT: CAUTION (単推しと複数推しでLTVに大きな差がありません)")


# ==========================================
# 分析4: 季節性の遷移 (accessed_at ベース)
# ==========================================
print("\n--- [Analysis 4] Seasonal Category Transition ---")

# 日付順にソート
df_sorted = df_purchase.sort_values(['user_id', 'accessed_at'])

# 月カラムの作成
df_sorted['month'] = df_sorted['accessed_at'].dt.month

# 次に買った商品のカテゴリを取得
df_sorted['next_category'] = df_sorted.groupby('user_id')['item_category_name'].shift(-1)

# 「夏(4-8月)にT-shirtを買った人」の次の購入
summer_tshirt_users = df_sorted[
    (df_sorted['month'].between(4, 8)) & 
    (df_sorted['item_category_name'] == 'T-shirt')
]

# 遷移先を集計
if len(summer_tshirt_users) > 0:
    transition_probs = summer_tshirt_users['next_category'].value_counts(normalize=True)
    print("Transition from Summer T-shirt:")
    print(transition_probs.head())
    
    # 判断: ランダム(20%)より明らかに高い遷移先があるか
    top_prob = transition_probs.iloc[0]
    if top_prob > 0.3:
        print(">> JUDGMENT: GO (特定のカテゴリへの遷移傾向が見られ、RNN等の時系列モデルが有効です)")
    else:
        print(">> JUDGMENT: NEUTRAL (次の購入カテゴリは分散しています)")
else:
    print("データ不足のため季節性分析をスキップしました")

In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from implicit.als import AlternatingLeastSquares
import warnings

# 警告の抑制（バージョンによるFutureWarning対策）
warnings.filterwarnings('ignore')

# ---------------------------------------------------------
# 1. データの準備 (ダミーデータ生成)
# ※実際はここに csv 読み込み処理を入れてください: df = pd.read_csv('data.csv')
# ---------------------------------------------------------

# カラム定義（ご提示いただいたデータセットに準拠）
data = {
    'user_id': [1, 1, 1, 2, 2, 3, 3, 3, 4, 1],
    'product_id': [101, 102, 103, 101, 104, 105, 106, 105, 107, 101],
    'event_action': [
        'view', 'add_to_cart', 'purchase',  # User 1
        'view', 'view',                     # User 2
        'purchase', 'favorite', 'view',     # User 3
        'view',                             # User 4
        'view'                              # User 1 (再閲覧)
    ],
    # 他のカラムは今回の計算には使用しませんが、存在するものとします
    'accessed_at': pd.date_range(start='2024-01-01', periods=10),
    'price': [3000] * 10
}

df = pd.DataFrame(data)

print("--- 元データ（先頭5行） ---")
print(df[['user_id', 'product_id', 'event_action']].head())
print("-" * 30)

# ---------------------------------------------------------
# 2. アクションの重み付け (Score Mapping)
# ---------------------------------------------------------
# viewを有効活用しつつ、purchaseの重要度を高く設定します。
# remove_from_cart は興味がなくなったと判断し、マイナススコアで相殺します。

event_weights = {
    'purchase': 10,          # 購入（最強のシグナル）
    'checkout': 8,           # 購入直前
    'add_to_cart': 5,        # 強い興味
    'add_to_wishlist': 3,    # 保存
    'favorite': 3,           # いいね
    'view': 1,               # 閲覧（弱いシグナルだが数が重要）
    'remove_from_cart': -5   # カート削除（興味の減衰）
}

# event_action を数値スコアに変換
df['event_score'] = df['event_action'].map(event_weights).fillna(0)

# ---------------------------------------------------------
# 3. ユーザー×プロダクトごとのスコア集計 (Aggregation)
# ---------------------------------------------------------
# 同じユーザーが同じプロダクトに対して行ったアクションを合計します。
# 例: view(1) * 5回 + add_to_cart(5) = 10点
grouped_df = df.groupby(['user_id', 'product_id'])['event_score'].sum().reset_index()

# マイナスになったスコア（カート削除などで）は0（無関心）としてクリッピング
grouped_df['event_score'] = grouped_df['event_score'].clip(lower=0)

# スコアが0のものは除外（疎行列化のため）
grouped_df = grouped_df[grouped_df['event_score'] > 0]

# ---------------------------------------------------------
# 4. 対数変換 (Log Transformation) - 重要！
# ---------------------------------------------------------
# 「ある一人のユーザーの大量購入」や「異常な閲覧回数」によるバイアスを抑える処理です。
# 生の回数 N を log(1 + N) に変換し、極端な値をマイルドにします。
grouped_df['event_score'] = np.log1p(grouped_df['event_score'])

# ---------------------------------------------------------
# 5. IDのマッピング (Encoding)
# ---------------------------------------------------------
# user_id, product_id を 0 から始まる連番（インデックス）に変換
user_ids = grouped_df['user_id'].unique()
product_ids = grouped_df['product_id'].unique()

user_to_index = {uid: i for i, uid in enumerate(user_ids)}
product_to_index = {pid: i for i, pid in enumerate(product_ids)}

# 逆引き用辞書（推薦結果を元のIDに戻すため）
index_to_user = {i: uid for i, uid in enumerate(user_ids)}
index_to_product = {i: pid for i, pid in enumerate(product_ids)}

grouped_df['user_index'] = grouped_df['user_id'].map(user_to_index)
grouped_df['product_index'] = grouped_df['product_id'].map(product_to_index)

# ---------------------------------------------------------
# 6. 疎行列 (CSR Matrix) の作成
# ---------------------------------------------------------
# implicitライブラリ用の行列作成 (rows=user, cols=item)
sparse_user_item = sparse.csr_matrix(
    (grouped_df['event_score'].astype(float),
     (grouped_df['user_index'], grouped_df['product_index'])),
    shape=(len(user_ids), len(product_ids))
)

# ---------------------------------------------------------
# 7. モデル学習 (ALS with Implicit Feedback)
# ---------------------------------------------------------
# factors: 次元数（特徴量の数）。データが多い場合は 64~128 程度に増やす。
# alpha: 信頼度係数。これが高いほど「アクションした事実」を重く見る。
model = AlternatingLeastSquares(
    factors=32,
    regularization=0.1,
    iterations=20,
    alpha=15,
    random_state=42
)

# 学習実行
# sparse_user_item 行列を渡します
model.fit(sparse_user_item)

# ---------------------------------------------------------
# 8. 推薦の実行と結果表示
# ---------------------------------------------------------
def recommend_for_user(target_user_id, n_items=10):
    if target_user_id not in user_to_index:
        print(f"User ID {target_user_id} は学習データに存在しません（コールドスタート）。")
        return

    user_idx = user_to_index[target_user_id]
    
    # 推薦実行
    # filter_already_liked_items=True: 既にアクションした商品は除外（新規発見を優先）
    # Falseにすると「リピート購入」も推薦に含まれるようになります
    ids, scores = model.recommend(
        user_idx, 
        sparse_user_item[user_idx], 
        N=n_items, 
        filter_already_liked_items=True
    )
    
    print(f"\nUser ID: {target_user_id} への推薦結果:")
    for i, score in zip(ids, scores):
        original_product_id = index_to_product[i]
        print(f" - Product ID: {original_product_id} (Score: {score:.4f})")

# テスト実行（User ID 1 に対して）
recommend_for_user(target_user_id=1, n_items=10)